In [1]:
import pandas as pd 
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
worktime = pd.read_csv('_SELECT_e_DeliveryTime_e_PickEmpName_sumPickQty_sku_count_sum_re_202104211136.csv')

In [6]:
worktime['DeliveryTime']=pd.to_datetime(worktime['DeliveryTime']).dt.normalize()

In [7]:
worktime.dtypes

DeliveryTime       datetime64[ns]
PickEmpName                object
sumPickQty                float64
sku_count                   int64
sum_resttime                int64
worktime                    int64
actual_worktime             int64
dtype: object

In [8]:
worktime.head()

,DeliveryTime,PickEmpName,sumPickQty,sku_count,sum_resttime,worktime,actual_worktime
0,2020-11-02,丁小顺,1329.0,16,1744,13817,12073
1,2020-11-03,丁小顺,1553.0,14,1738,15071,13333
2,2020-11-04,丁小顺,1108.0,14,1236,14525,13289
3,2020-11-05,丁小顺,1795.0,16,1617,15436,13819
4,2020-11-08,丁小顺,1557.0,16,1834,14499,12665


In [9]:
picking_product = pd.read_csv('pick_product.csv')

In [10]:
picking_product.head()

,area_id,area_sku,sku,PickEmpID,PickEmpName,product_name,product_spec,category_id,category_name,parent_category_id,parent_category_name,vendor_code,DeliveryDate,sale_unit_qty,sale_stock_qty,stock_qty
0,101,592797,1232303,11818,伍花姣,滋冠 日京蛋仔饼 葱油味 100g/包 4包/组,葱油味 100g/包 4包/组,715,13003饼干,712,13饼干糖果,18560,2020-11-01,4,1,30
1,101,528970,1232034,11818,伍花姣,阳光初见 蜂蜜面包 1kg/箱 正负20g 约22个/箱,1kg/箱 正负20g 约22个/箱,719,13007面包,712,13饼干糖果,11330,2020-11-01,1,1,1
2,101,556916,1232266,11818,伍花姣,牛氏庄园 牛轧糖 500g/袋 约61个/袋,500g/袋 约61个/袋,720,13008软糖,712,13饼干糖果,17257,2020-11-01,1,1,30
3,101,556916,1232266,11818,伍花姣,牛氏庄园 牛轧糖 500g/袋 约61个/袋,500g/袋 约61个/袋,720,13008软糖,712,13饼干糖果,17257,2020-11-01,1,1,30
4,101,564626,1231044,11818,伍花姣,俊媳妇 鸭翅根 盐焗味 40g/包 10包/袋,盐焗味 40g/包 10包/袋,687,11013盐焗零食,675,11休闲零食,14257,2020-11-01,1,1,20


In [33]:
picking_product.dtypes

area_id                   int64
area_sku                  int64
sku                       int64
PickEmpID                 int64
PickEmpName              object
product_name             object
product_spec             object
category_id               int64
category_name            object
parent_category_id        int64
parent_category_name     object
vendor_code               int64
DeliveryDate             object
sale_unit_qty             int64
sale_stock_qty            int64
stock_qty                 int64
weight                  float64
dtype: object

In [11]:
def list_str_to_float(name): 
    return list(map(float,name))

In [47]:
def getWeight(product_name, sale_unit_qty, sale_stock_qty):
    weight = 0
    product_name = product_name.split(' 适合')[0]
    productList = product_name.split(' ')

    for pName in productList:
        if 'kg' in pName:
            reg = re.findall(r'\d+\.?\d*(?=kg)', pName)
            if len(reg) > 0:
                reg = list_str_to_float(reg)
                weight = float(np.max(reg)) * 2
        elif 'KG' in pName:
            reg = re.findall(r'\d+\.?\d*(?=kg)', pName)
            if len(reg) > 0:
                reg = list_str_to_float(reg)
                weight = float(np.max(reg)) * 2
        elif '斤' in pName:
            reg = re.findall(r'\d+\.?\d*(?=斤)', pName)
            if len(reg) > 0:
                reg = list_str_to_float(reg)
                weight = float(np.max(reg))
        elif '公斤' in pName:
            reg = re.findall(r'\d+\.?\d*(?=公斤)', pName)
            if len(reg) > 0:
                reg = list_str_to_float(reg)
                weight = float(np.max(reg)) * 2
        elif 'g' in pName:
            reg = re.findall(r'\d+\.?\d*(?=g)', pName)
            if len(reg) > 0:
                reg = list_str_to_float(reg)
                weight = float(np.max(reg)) / 500
        elif 'L' in pName:
            reg = re.findall(r'\d+\.?\d*(?=L)', pName)
            if len(reg) > 0:
                reg = list_str_to_float(reg)
                weight = float(np.max(reg)) * 2
        elif 'ml' in pName:
            reg = re.findall(r'\d+\.?\d*(?=ml)', pName)
            if len(reg) > 0:
                reg = list_str_to_float(reg)
                weight = float(np.max(reg)) / 500
        if weight>0:
            break
    weight = weight * sale_unit_qty * sale_stock_qty
    return weight

In [48]:
import re
picking_product['weight'] =picking_product.apply(lambda picking_product: getWeight(picking_product['product_name'],picking_product['sale_unit_qty'],picking_product['sale_stock_qty']),axis=1)

In [19]:
picking_product.head()

,area_id,area_sku,sku,PickEmpID,PickEmpName,product_name,product_spec,category_id,category_name,parent_category_id,parent_category_name,vendor_code,DeliveryDate,sale_unit_qty,sale_stock_qty,stock_qty,weight
0,101,592797,1232303,11818,伍花姣,滋冠 日京蛋仔饼 葱油味 100g/包 4包/组,葱油味 100g/包 4包/组,715,13003饼干,712,13饼干糖果,18560,2020-11-01,4,1,30,0.80
1,101,528970,1232034,11818,伍花姣,阳光初见 蜂蜜面包 1kg/箱 正负20g 约22个/箱,1kg/箱 正负20g 约22个/箱,719,13007面包,712,13饼干糖果,11330,2020-11-01,1,1,1,2.00
2,101,556916,1232266,11818,伍花姣,牛氏庄园 牛轧糖 500g/袋 约61个/袋,500g/袋 约61个/袋,720,13008软糖,712,13饼干糖果,17257,2020-11-01,1,1,30,1.00
3,101,556916,1232266,11818,伍花姣,牛氏庄园 牛轧糖 500g/袋 约61个/袋,500g/袋 约61个/袋,720,13008软糖,712,13饼干糖果,17257,2020-11-01,1,1,30,1.00
4,101,564626,1231044,11818,伍花姣,俊媳妇 鸭翅根 盐焗味 40g/包 10包/袋,盐焗味 40g/包 10包/袋,687,11013盐焗零食,675,11休闲零食,14257,2020-11-01,1,1,20,0.08


In [49]:
picking_product.loc[picking_product.weight==0]

,area_id,area_sku,sku,PickEmpID,PickEmpName,product_name,product_spec,category_id,category_name,parent_category_id,parent_category_name,vendor_code,DeliveryDate,sale_unit_qty,sale_stock_qty,stock_qty,weight
185,101,516673,1230955,131345,刘宏,湘里山林 初生蛋30枚/盒,30枚,590,05003初产蛋,587,05蛋类,11920,2020-11-01,1,1,6,0.0
186,101,581092,1230686,131345,刘宏,咯咯达 玉米黄鸡蛋15枚/盒,15枚,588,05001鲜鸡蛋,587,05蛋类,19214,2020-11-01,1,1,20,0.0
189,101,516673,1230955,131345,刘宏,湘里山林 初生蛋30枚/盒,30枚,590,05003初产蛋,587,05蛋类,11920,2020-11-01,1,1,6,0.0
190,101,385439,1230762,131345,刘宏,龙丹 松花皮蛋20枚/盒,20枚,595,05008无铅松花蛋,587,05蛋类,13291,2020-11-01,1,1,12,0.0
191,101,581092,1230686,131345,刘宏,咯咯达 玉米黄鸡蛋15枚/盒,15枚,588,05001鲜鸡蛋,587,05蛋类,19214,2020-11-01,1,1,20,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484525,101,500139120,4013598,185267,钟畅,杉杉家纺 艾草抗菌驱蚊立体护颈枕芯 1个 约48X74cm,1个 约48X74cm,896,25010枕头,888,25家用纺织,11922,2021-04-14,1,1,50,0.0
484526,101,500158807,4013776,185267,钟畅,悦自然 任意剪安全防狼袜 1条 适合90-130斤,1条 适合90-130斤,902,25017裤袜,888,25家用纺织,13514,2021-04-14,1,1,300,0.0
484529,101,500158807,4013776,185267,钟畅,悦自然 任意剪安全防狼袜 1条 适合90-130斤,1条 适合90-130斤,902,25017裤袜,888,25家用纺织,13514,2021-04-14,1,1,300,0.0
484714,101,500027722,4013739,188691,黄积安,21金维他 21金维她(2阶)牌多种维生素矿物质片 56片/瓶,56片/瓶,755,15012营养保健,744,15冲饮保健,430104200009,2021-04-14,1,1,50,0.0


In [50]:
picking_product[picking_product.isna().any(axis=1)]

,area_id,area_sku,sku,PickEmpID,PickEmpName,product_name,product_spec,category_id,category_name,parent_category_id,parent_category_name,vendor_code,DeliveryDate,sale_unit_qty,sale_stock_qty,stock_qty,weight


In [39]:
w

0